<a href="https://colab.research.google.com/github/CaptainO5/AI/blob/master/8Puzzle_A_star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import heapq as hq
from itertools import count

In [0]:
class Node:
  def __init__(self, state, parent, cost, action):
    self.state = state
    self.parent = parent
    self.g = cost
    self.a = action # Action applied to reach the state L, R, T or D

  # Heuristic: sum of manhattan distances of each tile.. blank included
  def h(self, goal):
    sum = 0
    for i in range(3):
      for j in range(3):
        m, n = np.where(goal == self.state[i][j])
        m, n = int(m), int(n)
        sum += abs(i - m) + abs(j - n)
    return sum

  def expand(self):
    i, j = np.where(self.state == 0)
    i, j = int(i), int(j) # Current position of the blank -> {0}
    children = []

    # Move the blank one step at a time
    if self.a != 'D' and i - 1 >= 0:
        s = self.state.copy()
        s[i][j] = s[i - 1][j]
        s[i - 1][j] = 0
        children.append(Node(s, self, self.g + 1, 'T'))

    if self.a != 'R' and j - 1 >= 0:
        s = self.state.copy()
        s[i][j] = s[i][j - 1]
        s[i][j - 1] = 0
        children.append(Node(s, self, self.g + 1, 'L'))
    
    if self.a != 'L' and j + 1 <= 2:
        s = self.state.copy()
        s[i][j] = s[i][j + 1]
        s[i][j + 1] = 0
        children.append(Node(s, self, self.g + 1, 'R'))

    if self.a != 'T' and i + 1 <= 2:
        s = self.state.copy()
        s[i][j] = s[i + 1][j]
        s[i + 1][j] = 0
        children.append(Node(s, self, self.g + 1, 'D'))
    return children

In [0]:
def A_star(start, goal):
  openSet = []
  closed = []
  unique = count()

  f = start.h(goal)
  hq.heappush(openSet, (f, -1, start))
  while openSet:
    f_current, t, current = hq.heappop(openSet)
    if np.allclose(goal, current.state):
      print("Solution found")
      return current
    for child in current.expand():
      f_child = child.g + child.h(goal)

      # Check if in closed list
      for N in closed:
        if np.allclose(child.state, N):
          continue
      
      # Check if in open list
      for N in openSet:
        if np.allclose(child.state, N[2].state) and f_child < N[0]:
          hq.heapreplace(openSet, (f_child, N[1], child))
          continue
      
      hq.heappush(openSet, (f_child, next(unique), child))
    closed.append(current.state)

  print("Solution not found")

In [0]:
def print_soln(sol):
  if sol.parent == None:
    return
  print_soln(sol.parent)
  print(sol.state, "\n")

In [28]:
puzzle = np.asarray([[7, 2, 4], [5, 0, 6], [8, 3, 1]])

goal = list(np.arange(1, 9))
goal.append(0)
goal = np.asarray(goal).reshape(3, 3)

start = Node(puzzle, None, 0, '')

print_soln(A_star(start, goal))

Solution found
[[7 2 4]
 [5 3 6]
 [8 0 1]] 

[[7 2 4]
 [5 3 6]
 [8 1 0]] 

[[7 2 4]
 [5 3 0]
 [8 1 6]] 

[[7 2 4]
 [5 0 3]
 [8 1 6]] 

[[7 2 4]
 [0 5 3]
 [8 1 6]] 

[[0 2 4]
 [7 5 3]
 [8 1 6]] 

[[2 0 4]
 [7 5 3]
 [8 1 6]] 

[[2 4 0]
 [7 5 3]
 [8 1 6]] 

[[2 4 3]
 [7 5 0]
 [8 1 6]] 

[[2 4 3]
 [7 0 5]
 [8 1 6]] 

[[2 4 3]
 [7 1 5]
 [8 0 6]] 

[[2 4 3]
 [7 1 5]
 [0 8 6]] 

[[2 4 3]
 [0 1 5]
 [7 8 6]] 

[[2 4 3]
 [1 0 5]
 [7 8 6]] 

[[2 0 3]
 [1 4 5]
 [7 8 6]] 

[[0 2 3]
 [1 4 5]
 [7 8 6]] 

[[1 2 3]
 [0 4 5]
 [7 8 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 

